In this notebook I will explore and augment a custom dataset for entity recognition of cryptocurrencies scraped from reddit using my other project Dragonfly.

https://huggingface.co/datasets/conll2003
https://www.freecodecamp.org/news/getting-started-with-ner-models-using-huggingface/

https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_ner.ipynb#scrollTo=IV72GFgq_ZYb

https://github.com/nlpyang/BertSum

https://github.com/abhimishra91/transformers-tutorials

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import tqdm
from typing import List
from transformers import AutoTokenizer, BertForTokenClassification
from dotenv import load_dotenv
load_dotenv()
from pymongo import MongoClient
client = MongoClient(os.getenv("MONGO_CONNECTION_STRING"))
db = client["dragonfly"]

import torch
from torch.utils.data import Dataset, DataLoader
from train.dragonfly_dataset.src import DragonflyDataset

In [ ]:
# hyperparameters
MODEL_NAME = "distilbert-base-cased"
SEQ_MAX_LENGTH = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 2e-05
# i have a gpu for ants so not even distilbert fits :(
device = 'cpu' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
dt = DragonflyDataset(tokenizer, SEQ_MAX_LENGTH)
dt.__len__()

In [ ]:
class BERT_ner(torch.nn.Module):
    def __init__(self):
        super(BERT_ner, self).__init__()
        self.l1 = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=3)
        # self.l2 = torch.nn.Dropout(0.3)
        # self.l3 = torch.nn.Linear(768, 200)
    
    def forward(self, ids, mask, labels):
        output_1= self.l1(ids, mask, labels = labels)
        # output_2 = self.l2(output_1[0])
        # output = self.l3(output_2)
        return output_1
model = BERT_ner()
model.to(device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                #'num_workers': 0
                }
training_loader = DataLoader(dt, **train_params)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        targets = data['tags'].to(device, dtype = torch.long)

        loss = model(ids, mask, labels = targets)[0]

        # optimizer.zero_grad()
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #xm.optimizer_step(optimizer)
        #xm.mark_step() 

In [ ]:
for epoch in range(5):
    train(epoch)